In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout
from keras.layers.advanced_activations import LeakyReLU

In [12]:
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from numpy.random import seed
seed(1337)
from tensorflow.random import set_seed
set_seed(42)

from keras.applications import vgg16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.callbacks import ModelCheckpoint
from keras import layers, models, Model, optimizers

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from PIL import Image

In [4]:
relevant_categories = ['T-Shirts', 'Hoodies & Sweatshirts', 'Ankle Boots', 'Skirts',
 'Pants & Capris', 'Blouses & Shirts', 'Dresses', 'Jeans', 'Tank Tops',
 'Leggings', 'Casual Shirts', 'Polo', 'Shorts', 'Pullovers', 'Sweatpants',
 'Jumpsuits', 'Casual Pants']
nb_categories = len(relevant_categories)

In [5]:
def make_model():
    """
    Define your model architecture here.
    Returns Sequential model.
    """
    model = Sequential()

    model.add(Conv2D(kernel_size=(3,3), filters=16, input_shape=(32, 32, 3), padding="same"))
    model.add(LeakyReLU(0.1))
    model.add(Conv2D(kernel_size=(3,3), filters=32, input_shape=(32, 32, 3), padding="same"))
    model.add(LeakyReLU(0.1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(kernel_size=(3,3), filters=32, input_shape=(32, 32, 3), padding="same"))
    model.add(LeakyReLU(0.1))
    model.add(Conv2D(kernel_size=(3,3), filters=64, input_shape=(32, 32, 3), padding="same"))
    model.add(LeakyReLU(0.1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(LeakyReLU(0.1))
    model.add(Dropout(0.5))
    model.add(Dense(nb_categories))
    model.add(Activation("softmax"))
    
    return model

In [8]:
import keras

In [30]:
INIT_LR = 5e-3  # initial learning rate
BATCH_SIZE = 32
EPOCHS = 10

# don't call K.set_learning_phase() !!! (otherwise will enable dropout in train/test simultaneously)
model = make_model()  # define our model

# prepare model for fitting (loss, optimizer, etc)
model.compile(
    loss='categorical_crossentropy',  # we train 10-way classification
    optimizer=keras.optimizers.Adam(lr=INIT_LR),  # for SGD
    metrics=['accuracy']  # report accuracy during training
)

In [31]:
BATCH_SIZE = 32
checkpoint = ModelCheckpoint("sign_classifier.h5", monitor = 'val_acc', verbose=1, save_best_only=True, 
                             save_weights_only=False, mode='auto')
img_height, img_width = 32, 32

train_data_dir = "data/train"
val_data_dir = "data/val"
test_data_dir = "data/test"

train_datagen =  ImageDataGenerator(
    rescale=1./255
)
test_datagen =  ImageDataGenerator(
    rescale=1./255
)

# these are generators for train/test data that will read pictures #found in the defined subfolders of 'data/'
print('Total number of images for "training":')
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_height, img_width),
    batch_size = BATCH_SIZE, 
    class_mode = "categorical",
    shuffle=True
)
print('Total number of images for "validation":')
val_generator = test_datagen.flow_from_directory(
    val_data_dir,
    target_size = (img_height, img_width),
    batch_size = BATCH_SIZE,
    class_mode = "categorical",
    shuffle=True
)
print('Total number of images for "testing":')
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size = (img_height, img_width),
    batch_size = BATCH_SIZE,
    class_mode = "categorical",
    shuffle=True
)

Total number of images for "training":
Found 46451 images belonging to 17 classes.
Total number of images for "validation":
Found 5805 images belonging to 17 classes.
Total number of images for "testing":
Found 5815 images belonging to 17 classes.


In [28]:
def my_gen(gen):
    while True:
        try:
            data, labels = next(gen)
            yield data, labels
        except:
            pass

In [32]:
model.fit(
    my_gen(train_generator),  # prepared data
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=my_gen(val_generator),
    callbacks=[checkpoint],
    shuffle=True,
    verbose=1,
    steps_per_epoch=1435
)

Epoch 1/10
1435/1435 [==============================] - ETA: 0s - loss: 2.0430 - accuracy: 0.3572

KeyboardInterrupt: 